In [7]:
import time
from datetime import date
import pandas as pd
import os
import glob
import pycytominer
import sys
import CBE_utils as CBE
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.spatial.distance import correlation
import re
import gc

In [8]:
import importlib
importlib.reload(CBE)

<module 'CBE_utils' from '/home/schmiedc/FMP_Docs/Projects/ECBL_Project/QualityControl_analysis_revision/notebooks_revision/CBE_utils.py'>

In [9]:
#input_path = "/home/schmiedc/FMP_Docs/Projects/ECBL_Project/QualityControl_analysis_revision/output/"
# annotation_dir = "/home/schmiedc/FMP_Docs/Projects/ECBL_Project/QualityControl_analysis_revision/annotation/"

In [10]:
input_path = "/home/schmiedc/FMP_Docs/Projects/ECBL_Project/QualityControl_analysis_revision/tech_test_output/"
annotation_dir = "/home/schmiedc/FMP_Docs/Projects/ECBL_Project/QualityControl_analysis_revision/annotation/"

# test
input_path = "/home/schmiedc/FMP_Docs/Projects/ECBL_Project/QualityControl_analysis_revision/test_output/"
annotation_dir = "/home/schmiedc/FMP_Docs/Projects/ECBL_Project/QualityControl_analysis_revision/test_annotation/"

In [11]:
pattern = "[A-Z]*_R[1-4].csv"

file_list = glob.glob(input_path + os.sep + '*' + os.sep + '*' + os.sep + pattern, recursive=True)

In [12]:
for file in file_list:
    
    plate_map_name = os.path.splitext(os.path.basename(file))[0]
    
    try:
        
        dataframe = pd.read_csv(file)
        row_count = dataframe.shape[0]
        
        print(f"INFO: {plate_map_name} has {row_count} rows")
        
        
    except Exception as e:
        
        print(f"Error reading file {plate_map_name}: {e}")   
        
    features = CBE.get_feature_vector(dataframe)
    metadata_dataframe = set(dataframe.columns) - set(features)
        
    ### convert sets to list
    features_list = list(features)
    metadata_list = list(metadata_dataframe)
        
    number_features = len(features_list)
        
    print(f"INFO: {plate_map_name} has {number_features} features")
        
    normalizer = "mad_robustize" 
    
    dataframe_normalized = pycytominer.normalize(
        
        profiles = dataframe,
        features = features_list,
        meta_features = metadata_list,
        method = normalizer, ### Method to normalize the data
        samples = "Metadata_RoughID == 'DMSO'" # normalization performed on neg. controls
    )
    
    print(f"INFO: Normalized with {normalizer}")
    
    filename_norm_data = plate_map_name + "_" + normalizer + ".csv"
    
    path_norm_data = os.path.dirname(file)
    file_path_norm_data = os.path.join(path_norm_data, filename_norm_data)
    
    dataframe_normalized.to_csv(file_path_norm_data, index=False)
    
    print(f"INFO: Saved {filename_norm_data}")

INFO: C1223_R4 has 384 rows
INFO: C1223_R4 has 2977 features
INFO: Normalized with mad_robustize
INFO: Saved C1223_R4_mad_robustize.csv
INFO: C1227_R3 has 384 rows
INFO: C1227_R3 has 2977 features
INFO: Normalized with mad_robustize
INFO: Saved C1227_R3_mad_robustize.csv
INFO: C1220_R2 has 384 rows
INFO: C1220_R2 has 2977 features
INFO: Normalized with mad_robustize
INFO: Saved C1220_R2_mad_robustize.csv
INFO: USC01_R1 has 384 rows
INFO: USC01_R1 has 2977 features
INFO: Normalized with mad_robustize
INFO: Saved USC01_R1_mad_robustize.csv
INFO: USC02_R1 has 384 rows
INFO: USC02_R1 has 2977 features
INFO: Normalized with mad_robustize
INFO: Saved USC02_R1_mad_robustize.csv
